# 6 - pySAM Simulation

<ol>
    <li> <a href='#step1'> Create default PV </a></li>
    <li> <a href='#step2'> Set values that should not be default to represent our system </a></li>
    <li> <a href='#step3'> Assign Solar Resource </a></li>
    <li> <a href='#step4'> Obtaining the MODULE Data </a></li>
    <ul> <li> <a href='#step4a'> Get File through PVLib </a></li>
        <li> <a href='#step4b'> Get File from the SAM repository URL </a></li></ul>
    <li> <a href='#step5'> Run, Save and Explore Outputs </a></li>
    </ol> <li> <a href='#stepExtra1'> Extra: Comparison of pySAM Default vs SAM GUI exported values </a></li>
        

In [ ]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install -r https://raw.githubusercontent.com/PVSC-Python-Tutorials/pyData-2021-Solar-PV-Modeling/main/requirements.txt

In [1]:
datafolder = r'data'
exampleflag = False
debugflag = False

In [ ]:
import PySAM.Pvsamv1 as pv
import PySAM
import xlsxwriter
import json
import pandas as pd
import os
import pprint as pp


In [3]:
PySAM.__version__

'2.2.4'

<a id='step1'></a>

# 1. Create default PV 

Doing Commercial type because the size is bigger and it's tracking than a rooftop. We are not really interested in the financial model here though just in the irradiance and power generation so you can choose a different one if you want 

In [4]:
sam1 = pv.default("FlatPlatePVCommercial")

You can explore attributes of the pySAM object with the below cell. This will give variables, methods, etc.

In [5]:
#dir(sam1)

You can also query a single attribute. And export it to see the contents more easily 

In [6]:
sam1.__getattribute__('SolarResource').use_wf_albedo
dic = sam1.export()
dic

{'SolarResource': {'albedo': (0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2),
  'irrad_mode': 0.0,
  'sky_model': 2.0,
  'use_wf_albedo': 0.0},
 'Losses': {'acwiring_loss': 1.0,
  'dcoptimizer_loss': 0.0,
  'en_snow_model': 0.0,
  'subarray1_dcwiring_loss': 2.0,
  'subarray1_diodeconn_loss': 0.5,
  'subarray1_mismatch_loss': 2.0,
  'subarray1_nameplate_loss': 0.0,
  'subarray1_rear_irradiance_loss': 0.0,
  'subarray1_soiling': (5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0),
  'subarray1_tracking_loss': 0.0,
  'subarray2_dcwiring_loss': 2.0,
  'subarray2_diodeconn_loss': 0.5,
  'subarray2_mismatch_loss': 2.0,
  'subarray2_nameplate_loss': 0.0,
  'subarray2_rear_irradiance_loss': 0.0,
  'subarray2_soiling': (5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0),
  'subarray2_tracking_loss': 0.0,
  'subarray3_dcwiring_loss': 2.0,
  'subarray3_diodeconn_loss':

Or save it as a csv if you want to explore the categories and variables

In [7]:
filesave = os.path.join(datafolder,'JSON_Default.xlsx')

workbook = xlsxwriter.Workbook(filesave)
worksheet = workbook.add_worksheet()
row = 0
col = 0
order=sorted(dic.keys())
for key in order:
    row += 1
    worksheet.write(row, col,     key)
    for item in dic[key]:
        worksheet.write(row, col + 1, item)
        worksheet.write(row, col + 2, str(dic[key][item]))

        row += 1

workbook.close()

<a id='step2'></a>

<a id='step2'></a>

# 2.  Set values that should not be default to represent our system:

In [8]:
solar_resource_file = r'data/SAM Downloaded Weather Files/SRRL_WeatherFile_SAM_60_2020.csv'
albedo = [0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224]

module_aspect_ratio = 2

subarray1_track_mode = 1
subarray1_backtrack = 1
subarray1_rotlim = 50

subarray1_gcr = 0.34903
subarray1_modules_per_string = 20
subarray1_nstrings = 10
subarray1_nmodx = 20
subarray1_nmody = 1
subarray1_shade_mode = 1

inverter_count = 10

subarray1_rear_irradiance_loss = 10
subarray1_soiling = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
use_wf_albedo = 1
# cec_a_ref = 1.885731
# inv_snl_vdco = 310
#  part of Inverter mppt_low_inverter = 100
subarray1_tilt = 0 


In [9]:
newval = { 'SolarResource': {
                            'albedo': albedo,
                            'use_wf_albedo': use_wf_albedo},
           'SystemDesign' : {
                            'inverter_count':inverter_count,
                            'subarray1_backtrack':subarray1_backtrack,
                            'subarray1_gcr':subarray1_gcr,
                            'subarray1_modules_per_string':subarray1_modules_per_string,
                            'subarray1_nstrings':subarray1_nstrings,
                            'subarray1_rotlim':subarray1_rotlim,
                            'subarray1_track_mode':subarray1_track_mode,
                            'subarray1_tilt': subarray1_tilt},
            'Layout':      {
                            'module_aspect_ratio': module_aspect_ratio,
                            'subarray1_nmodx':subarray1_nmodx,
                            'subarray1_nmody':subarray1_nmody},
            'Shading' :    {'subarray1_shade_mode':subarray1_shade_mode},
            'Losses' :     {
                            'subarray1_soiling':subarray1_soiling,
                            'subarray1_rear_irradiance_loss':subarray1_rear_irradiance_loss},
             }



We are going to use the method <b>'assign'</b>, becasue that only replaces the values we are providing. <b>'replace'</b> replaces all the internally generated dictionary entries with only the ones we are providing. 

In [10]:
sam1.assign(newval)

<a id='step3'></a>

# 3. Assign Solar Resource

No solar resource is assigned by default. If this is not assigned the simulation will not run.

In [11]:
sam1.SolarResource.solar_resource_file = solar_resource_file

<a id='step4'></a>

# 4. Obtaining the MODULE Data

Two methods are explored: 
    1. Using PVLib
    2. getting SAM's CEC library URL and reading the csv.

<a id='step4a'></a>

### 4A. Using PVLib

In [12]:
import pvlib
CECMODS = pvlib.pvsystem.retrieve_sam(name='CECMod')

# the CEC modules are a pandas DataFrame oriented as columns, transpose to arrange
# as indices
CECMODS.T.head()

,Technology,Bifacial,STC,PTC,A_c,Length,Width,N_s,I_sc_ref,V_oc_ref,...,a_ref,I_L_ref,I_o_ref,R_s,R_sh_ref,Adjust,gamma_r,BIPV,Version,Date
A10Green_Technology_A10J_S72_175,Mono-c-Si,0,175.0914,151.2,1.3,1.576,0.825,72,5.17,43.99,...,1.981696,5.175703,0.0,0.316688,287.102203,16.057121,-0.5072,N,SAM 2018.11.11 r2,1/3/2019
A10Green_Technology_A10J_S72_180,Mono-c-Si,0,179.928,155.7,1.3,1.576,0.825,72,5.31,44.06,...,1.988414,5.316148,0.0,0.299919,259.047943,16.418983,-0.5072,N,SAM 2018.11.11 r2,1/3/2019
A10Green_Technology_A10J_S72_185,Mono-c-Si,0,184.7016,160.2,1.3,1.576,0.825,72,5.43,44.14,...,1.984817,5.435676,0.0,0.311962,298.424438,15.688233,-0.5072,N,SAM 2018.11.11 r2,1/3/2019
A10Green_Technology_A10J_M60_220,Multi-c-Si,0,219.876,189.1,1.624,1.632,0.995,60,7.95,36.06,...,1.673094,7.959062,0.0,0.140393,123.168404,21.875164,-0.5196,N,SAM 2018.11.11 r2,1/3/2019
A10Green_Technology_A10J_M60_225,Multi-c-Si,0,224.9856,193.5,1.624,1.632,0.995,60,8.04,36.24,...,1.671782,8.047206,0.0,0.14737,164.419479,20.698376,-0.5196,N,SAM 2018.11.11 r2,1/3/2019


The Module we want for this simulation has this name in SAM: <b> "Prism Solar Technologies_ Inc. Bi72-457BSTC" </b>. The name should be written similar with underscores, but is not always teh same in the database. To make it easier, we filter by keywords.

In [13]:
cs_220m_mods = CECMODS.T.index.str.startswith('Prism') & CECMODS.T.index.str.contains('457BSTC')
mymod = CECMODS.T[cs_220m_mods]
mymod.T

,Prism_Solar_Technologies_Bi72_457BSTC
Technology,Multi-c-Si
Bifacial,1
STC,360.214
PTC,327.6
A_c,1.979
Length,2.011
Width,0.984
N_s,72
I_sc_ref,9.8
V_oc_ref,48.2


Asigning variables about the bifaciality aspect of our site. This are saved inside the <b> 'CECPerformanceModelWithModuleDatabase' </b> but are not in the CEC table, they are default/user provided

## B. Get File from the SAM repository URL

In [14]:
# Testing with a local file first downloaded. Please not that 'download' from Github web interface messes up the csv properties and this would fail.
fileee= r'C:\Users\sayala\Documents\GitHub\RTCanalysis\BEST_Sam_PVsyst_BR_BVF_Results\pySAM\CEC Modules.csv'

df = pd.read_csv(fileee, index_col=0)
mymod = df.index.str.startswith('Prism') & df.index.str.contains('457BSTC')
df[mymod].T

Name,Prism Solar Technologies_ Inc. Bi72-457BSTC
Manufacturer,Prism Solar Technologies_ Inc.
Technology,Multi-c-Si
Bifacial,1
STC,360.214
PTC,327.6
A_c,1.979
Length,2.011
Width,0.984
N_s,72
I_sc_ref,9.8


In [15]:
import requests

The URL you see when you navigate to the github looks like:

    >> https://github.com/NREL/SAM/blob/master/deploy/libraries/CEC%20Modules.csv
    
However, to access it you must rename the github.com to raw.githubusercontent.com, and also remove the 'blob' part, so

    >> https://raw.githubusercontent.com/NREL/SAM/master/deploy/libraries/CEC%20Modules.csv'


In [16]:
url = 'https://raw.githubusercontent.com/NREL/SAM/master/deploy/libraries/CEC%20Modules.csv'
df = pd.read_csv(url, index_col=0)
modfilter = df.index.str.startswith('Prism') & df.index.str.contains('457BSTC')
mymod = df[modfilter]


<div class="alert alert-block alert-warning">
<b>Note:</b> As of Oct 7th, the SAM GUI version corresponds to the 'patch' branch. This has a more updated version of the CEC csv. So we are atually going to use that one by repeating the above procedure but pointing to that branch. This might not be necessary or might change later on
</div>


In [17]:
masterdate = mymod['Date'][0]

In [18]:
#url = 'https://raw.githubusercontent.com/NREL/SAM/develop/deploy/libraries/CEC%20Modules.csv'
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
df = pd.read_csv(url, index_col=0)
modfilter = df.index.str.startswith('Prism') & df.index.str.contains('457BSTC')
mymod = df[modfilter]
otherbranchdate = mymod['Date'][0]

In [19]:
print("Master database is dated:", masterdate)
print("Patch database is dated:", otherbranchdate)

Master database is dated: 12/19/2019
Patch database is dated: 11/25/2020


#### Assign the Variables to the SAM object from the CEC data

This are read as strings, so we need to make 

In [20]:
float(mymod.Bifacial[0])

1.0

In [21]:
cec_a_ref = float(mymod.a_ref[0]) # 1.885731
cec_adjust = float(mymod.Adjust[0]) # 10.400029
cec_alpha_sc = float(mymod.alpha_sc[0]) # 0.004675
cec_area = float(mymod.A_c[0]) # 1.979

cec_beta_oc = float(mymod.beta_oc[0]) # -0.139925
cec_gamma_r = float(mymod.gamma_r[0]) # -0.4028
cec_i_l_ref = float(mymod.I_L_ref[0]) # 9.80468
cec_i_mp_ref = float(mymod.I_mp_ref[0]) # 9.26
cec_i_o_ref = float(mymod.I_o_ref[0]) # 7.72903e-11       # NOT THE SAME, DB says "0.0"
cec_i_sc_ref = float(mymod.I_sc_ref[0]) # 9.8
cec_is_bifacial = int(mymod.Bifacial[0]) # 1
cec_module_length = float(mymod.Length[0]) # 2.011
cec_module_width = float(mymod.Width[0]) # 0.984
module_aspect_ratio = cec_module_length/cec_module_width    # 2.043699187


cec_n_s = float(mymod.N_s[0]) # 72
cec_r_s = float(mymod.R_s[0]) #0.396799
cec_r_sh_ref = float(mymod.R_sh_ref[0]) # 830.975492
cec_t_noct = float(mymod.T_NOCT[0]) # 47.4
cec_v_mp_ref = float(mymod.V_mp_ref[0]) # 38.9
cec_v_oc_ref = float(mymod.V_oc_ref[0]) # 48.2


In [22]:
cec_bifacial_ground_clearance_height = 1.5
cec_bifacial_transmission_factor = 0
cec_bifaciality = 0.694 # WHY IS THIS NOT IN THE CEC Data?

# Temperature corrections -- We are not doing this. Should we?
# cec_temp_corr_mode = 1?
# cec_array_cols = 10 #    Already default
# cec_array_rows = 1  #    Already default
# ?? cec_temp_corr_mode	0       ???
# ?? cec_transient_thermal_model_unit_mass	11.0919


In [23]:
newval = { 'Layout':{'module_aspect_ratio':module_aspect_ratio},
            'CECPerformanceModelWithModuleDatabase': {
                'cec_a_ref': cec_a_ref,
                'cec_adjust': cec_adjust,
                'cec_alpha_sc': cec_alpha_sc,
                'cec_area': cec_area,
                'cec_beta_oc': cec_beta_oc,
                'cec_gamma_r': cec_gamma_r,
                'cec_i_l_ref': cec_i_l_ref,
                'cec_i_mp_ref': cec_i_mp_ref,
                'cec_i_o_ref': cec_i_o_ref,
                'cec_i_sc_ref': cec_i_sc_ref,
                'cec_is_bifacial': cec_is_bifacial,
                'cec_module_length': cec_module_length,
                'cec_module_width': cec_module_width,
                'cec_n_s': cec_n_s,
                'cec_r_s': cec_r_s,
                'cec_r_sh_ref': cec_r_sh_ref,
                'cec_t_noct': cec_t_noct,
                'cec_v_mp_ref': cec_v_mp_ref,
                'cec_v_oc_ref': cec_v_oc_ref,
                'cec_bifacial_ground_clearance_height': cec_bifacial_ground_clearance_height,
                'cec_bifacial_transmission_factor': cec_bifacial_transmission_factor,
                'cec_bifaciality': cec_bifaciality
             }}

sam1.assign(newval)

## Get Inverter Values

First let's download the database

In [24]:
#url = 'https://raw.githubusercontent.com/NREL/SAM/develop/deploy/libraries/CEC%20Modules.csv'
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Inverters.csv'
df = pd.read_csv(url, index_col=0)


Our inverter is 'Fronius USA: Fronius Symo 10.0-3 480 [480V]'. Using the same look-up technique as with the module

In [25]:
modfilter = df.index.str.startswith('Fronius USA') & df.index.str.contains('480V')  & df.index.str.contains('10.0')
myinv = df[modfilter]
myinv

,Vac,Pso,Paco,Pdco,Vdco,C0,C1,C2,C3,Pnt,Vdcmax,Idcmax,Mppt_low,Mppt_high,CEC_Date,CEC_Type,CEC_hybrid
Name,,,,,,,,,,,,,,,,,
Fronius USA: Fronius Symo 10.0-3 480 [480V],480,64.071510,9995,10259.992188,670,-2.753997e-07,-0.000022,-0.000375,0.001689,2.998500,800,15.313421,100,800,NaN,Utility Interactive,empty


In [26]:
inv_snl_c0 = float(myinv['C0'])
inv_snl_c1 = float(myinv['C1'])
inv_snl_c2 = float(myinv['C2'])
inv_snl_c3 = float(myinv['C3'])
inv_snl_paco = float(myinv['Paco'])
inv_snl_pdco = float(myinv['Pdco']) 
inv_snl_pnt = float(myinv['Pnt'])
inv_snl_pso = float(myinv['Pso'])
inv_snl_vdcmax = float(myinv['Vdcmax'])
inv_snl_vdco =  float(myinv['Vdco'])
mppt_low_inverter = float(myinv['Mppt_low'])
mppt_hi_inverter = float(myinv['Mppt_high'])

You can set an individual value with __setattr__

In [27]:
sam1.Inverter.__setattr__('inv_snl_paco', inv_snl_paco)

Or do them all together like we've been doing in dictionary format

In [28]:
newval = {  'Inverter':            { 'inv_snl_paco': inv_snl_paco,
                                   'mppt_low_inverter': mppt_low_inverter,
                                   'mppt_hi_inverter': mppt_hi_inverter},
            'InverterCECDatabase': {
                                    'inv_snl_c0': inv_snl_c0,
                                    'inv_snl_c1': inv_snl_c1,
                                    'inv_snl_c2': inv_snl_c2,
                                    'inv_snl_c3': inv_snl_c3,
                                    'inv_snl_paco': inv_snl_paco,
                                    'inv_snl_pdco': inv_snl_pdco,
                                    'inv_snl_pnt': inv_snl_pnt,
                                    'inv_snl_pso': inv_snl_pso,
                                    'inv_snl_vdcmax': inv_snl_vdcmax,
                                    'inv_snl_vdco': inv_snl_vdco
             }}

sam1.assign(newval)

# Assign VALUES From the GUI

Some values are calculated internally by the SAM GUI based on other values you provide. To identify this items, you can look in the documentation for the warning "Changes to this variable may require updating the values of the following:" or "This variable may need to be updated if the values of the following have changed".


<div class="alert alert-block alert-warning">
<b>Note:</b> There is no way for pySAM to calculate these updates internally at the moment. 
</div>

The easiest is to simulate your parameters so far in the GUI, and get the values from there. Alternatively, you could export the whole simulation as a JSON, load it and execute it. However for this tutorial we are starting from the default and modifying it here, so let's assign those values now.


In [29]:
inv_snl_eff_cec = 96.776
system_capacity =  72.04280090332031
inverter_count = 10
inv_tdc_cec_db = [[1, 52.79999923706055, -0.020999999716877937]] # Temperature derate curves for CEC Database [(Vdc, C, %/C)]

sam1.SystemDesign.__setattr__('system_capacity', system_capacity)
sam1.Inverter.__setattr__('inv_snl_eff_cec', inv_snl_eff_cec)
sam1.Inverter.__setattr__('inverter_count', inverter_count)
sam1.InverterCECDatabase.__setattr__('inv_tdc_cec_db', inv_tdc_cec_db)

# SAVING

In [30]:
sam1.execute()

In [31]:
foo = sam1.Outputs.export()

In [45]:
#foo['subarray1_poa_rear']
#foo['subarray1_dc_gross']
list(foo.keys())

['ac_lifetime_loss',
 'ac_loss',
 'ac_perf_adj_loss',
 'ac_transmission_loss',
 'ac_wiring_loss',
 'airmass',
 'alb',
 'annual_ac_gross',
 'annual_ac_inv_clip_loss_percent',
 'annual_ac_inv_eff_loss_percent',
 'annual_ac_inv_pnt_loss_percent',
 'annual_ac_inv_pso_loss_percent',
 'annual_ac_lifetime_loss_percent',
 'annual_ac_loss_ond',
 'annual_ac_perf_adj_loss_percent',
 'annual_ac_wiring_loss',
 'annual_ac_wiring_loss_percent',
 'annual_dc_diodes_loss',
 'annual_dc_diodes_loss_percent',
 'annual_dc_gross',
 'annual_dc_inv_tdc_loss_percent',
 'annual_dc_invmppt_loss',
 'annual_dc_lifetime_loss_percent',
 'annual_dc_loss_ond',
 'annual_dc_mismatch_loss',
 'annual_dc_mismatch_loss_percent',
 'annual_dc_module_loss_percent',
 'annual_dc_mppt_clip_loss_percent',
 'annual_dc_nameplate_loss',
 'annual_dc_nameplate_loss_percent',
 'annual_dc_net',
 'annual_dc_nominal',
 'annual_dc_optimizer_loss',
 'annual_dc_optimizer_loss_percent',
 'annual_dc_perf_adj_loss_percent',
 'annual_dc_snow_loss_

In [ ]:
filesave = os.path.join(datafolder,'pySAM_Outputs.xlsx')

workbook = xlsxwriter.Workbook(filesave)
worksheet = workbook.add_worksheet()

row=0
col=0

for key in foo.keys():
    row += 1
    worksheet.write(row, col, key)
    worksheet.write(row, col + 1, str(foo[key]))

workbook.close()

<a id='stepExtra1'></a>

# ::Extra:: Comparison of pySAM Default vs SAM GUI exported values

If you have a JSON that you exported from SAM, it will look a bit different than the default pySAM object you created. pySAM groups all variables grouped by different categories. The below code will save that so you can compare the values between those of a SAM simulation and this default. Your SAM JSON will also have more variables for the grid, and economic calculation tabs which we have not created in the pySAM object yet.

In [ ]:
# Prism File
Prismfile = os.path.join(datafolder,'Row2Prism.json')

with open(Prismfile) as f:
    dic = json.load(f)

In [ ]:
'''
filesave = os.path.join(datafolder,'JSON_Prism.xlsx')

workbook = xlsxwriter.Workbook(filesave)
worksheet = workbook.add_worksheet()

row=0
col=0

for key in dic.keys():
    row += 1
    worksheet.write(row, col, key)
    worksheet.write(row, col + 1, str(dic[key]))

workbook.close()
''';

In [ ]:
maincomparisonvars = [sam1.AdjustmentFactors,
 sam1.CECPerformanceModelWithModuleDatabase,
 sam1.Inverter,
 sam1.InverterCECDatabase,
 sam1.Layout,
 sam1.Lifetime,
 sam1.Load,
 sam1.Losses,
 sam1.MermoudLejeuneSingleDiodeModel,
 sam1.Module,
 sam1.Shading,
 sam1.SolarResource,
 sam1.SystemDesign]

In [ ]:
d4 = dict(maincomparisonvars[0].export())
for category in maincomparisonvars[1:]:
    d4.update(category.export())
    
pySAMdic = pd.DataFrame.from_dict(d4, orient='index')

In [ ]:
guiSAMdic = pd.DataFrame.from_dict(dic,orient='index')

In [46]:
guiSAMdic

NameError: name 'guiSAMdic' is not defined

In [ ]:
result = pd.concat([guiSAMdic, pySAMdic], axis=1, join="inner")
result.columns = ['guiSAM', 'pySAM']
result.to_csv(os.path.join(Resultsfolder, 'Side Comparison.csv'))